In [1]:
from google.colab import drive

# Google Drive 마운트
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
!pip install dtaidistance --quiet

import numpy as np
import pandas as pd
from dtaidistance import dtw
from sklearn.preprocessing import MinMaxScaler
from collections import Counter
import random

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 30.2 MB/s eta 0:00:00


In [42]:
df_X_train = pd.read_csv('/content/drive/My Drive/PhalangesOutlinesCorrect/X_train_Worms.csv')
df_y_train = pd.read_csv('/content/drive/My Drive/PhalangesOutlinesCorrect/y_train_Worms.csv')
df_X_test = pd.read_csv('/content/drive/MyDrive/PhalangesOutlinesCorrect/X_test_Worms.csv')
df_y_test = pd.read_csv('/content/drive/MyDrive/PhalangesOutlinesCorrect/y_test_Worms.csv')

In [43]:
# numpy 배열로 변환
X = df_X_train.values
y = df_y_train.values.ravel()  # 1차원 벡터로 변환
X_test = df_X_test.values
y_test = df_y_test.values.ravel()

In [44]:
df_X_train.head()

,0,1,2,3,4,5,6,7,8,9,...,890,891,892,893,894,895,896,897,898,899
0,1.729001,1.725254,1.721507,1.717760,1.713597,1.709850,1.706103,1.702356,1.698193,1.694446,...,-1.694462,-1.698209,-1.702373,-1.706120,-1.709866,-1.713613,-1.717777,-1.721524,-1.725271,-1.729018
1,-0.173428,-0.136345,-0.136345,-0.099261,-0.358847,-0.136345,-0.210512,-0.099261,-0.321763,-0.247596,...,-1.434276,-1.360108,-1.434276,-1.360108,-1.471360,-1.397192,-1.397192,-1.323025,-1.360108,-1.174690
2,-1.107453,-1.087120,-1.066786,-1.046452,-1.026119,-1.005785,-0.985451,-0.965118,-1.001718,-1.013918,...,1.678260,1.657927,1.633526,1.580659,1.511524,1.511524,1.515591,1.531858,1.552192,1.613193
3,-0.846077,-0.815577,-0.772877,-0.717977,-0.730177,-0.626477,-0.559377,-0.516677,-0.504477,-0.351977,...,-0.437377,-0.394677,-0.382477,-0.358077,-0.370277,-0.339777,-0.351977,-0.351977,-0.315377,-0.309277
4,0.745318,0.739891,0.734464,0.734464,0.729038,0.723611,0.718184,0.712758,0.712758,0.707331,...,2.145412,2.172546,2.145412,2.129132,2.129132,2.085718,2.026024,1.993464,1.971757,1.928343


In [45]:
len(df_X_train)

181

In [46]:
df_y_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181 entries, 0 to 180
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   0       181 non-null    int64
dtypes: int64(1)
memory usage: 1.5 KB


In [47]:
df_y_train

,0
0,1
1,1
2,1
3,1
4,1
...,...
176,5
177,5
178,5
179,5


In [48]:
df_y_train.value_counts()

,count
0,
1,76
4,32
2,31
3,25
5,17


In [49]:
def normalize(X):
    scaler = MinMaxScaler(feature_range=(-1, 1))
    return scaler.fit_transform(X), scaler

def compute_centroid(X):
    return np.mean(X, axis=0)

def compute_wpd(sample, center):
    return dtw.distance_fast(sample, center)

def classify_safe_points(X_class, center):
    wpds = np.array([compute_wpd(x, center) for x in X_class])
    wpd_average = np.mean(wpds)
    safe_points = X_class[wpds <= wpd_average]
    return safe_points, wpd_average

def interpolate_between_safe_and_center(safe, center):
    alpha = np.random.uniform(0, 1)
    return safe + alpha * (center - safe)

In [50]:
# 논문 수식 기반 증강 수 계산
def compute_target_counts(y, T=2):
    class_counts = Counter(y)
    classes = list(class_counts.keys())
    sum_classes = len(classes)
    N = len(y)
    class_aver = int((N * T) / sum_classes)

    # 증강 필요 없는 클래스 정보
    no_aug_classes = [c for c in classes if class_counts[c] >= class_aver]
    a = len(no_aug_classes)
    n = sum([class_counts[c] for c in no_aug_classes])

    # 최종 타겟 수 계산
    #클래스 c가 증강 대상이면 → 목표 수 target_num으로 설정
    #클래스 c가 증강 불필요이면 → 현재 수 유지
    target_counts = {}
    for c in classes:
        x = class_counts[c]
        if c in no_aug_classes:
            target_counts[c] = x  # 증강 X
        else:
            target_num = int(((N * T) - n) / (sum_classes - a))
            target_counts[c] = max(x, target_num)  # 현재보다 크면 증강
    return target_counts

In [51]:
def m_smote_full_v2(X_original, y_original, T=2):
    X_normalized, scaler = normalize(X_original)
    y = y_original
    classes = np.unique(y)
    target_counts = compute_target_counts(y, T)

    X_augmented = []
    y_augmented = []

    for label in classes:
        X_c = X_normalized[y == label]
        n_c = len(X_c)
        n_target = target_counts[label]

        if n_c >= n_target:
            continue  # 증강 필요 없음

        center = compute_centroid(X_c)
        safe_points, _ = classify_safe_points(X_c, center)

        if len(safe_points) == 0:
            print(f"[경고] 클래스 {label}에 safe point 없음 — 증강 생략")
            continue

        num_to_generate = n_target - n_c
        for _ in range(num_to_generate):
            x_safe = safe_points[random.randint(0, len(safe_points) - 1)]
            new_sample = interpolate_between_safe_and_center(x_safe, center)
            X_augmented.append(new_sample)
            y_augmented.append(label)

    X_all = np.vstack([X_normalized] + X_augmented)
    y_all = np.concatenate([y, y_augmented])
    return X_all, y_all, target_counts


In [52]:
X_synthetic, y_synthetic, target_counts = m_smote_full_v2(X, y, T=2)

len(X_synthetic)

360

In [53]:
import matplotlib.pyplot as plt
from scipy.spatial.distance import euclidean
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report, recall_score, confusion_matrix
from urllib.request import urlopen
from io import StringIO
from google.colab import drive
import random
from imblearn.metrics import specificity_score
from scipy.stats import mode
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Conv1D, Flatten
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from collections import defaultdict

In [54]:
def predict_with_logistic_regression(X_train, y_train, X_test):
    model = LogisticRegression()
    model.fit(X_train, y_train)
    return model.predict(X_test)

def predict_with_cart(X_train, y_train, X_test):
    model = DecisionTreeClassifier()
    model.fit(X_train, y_train)
    return model.predict(X_test)

def predict_with_knn(X_train, y_train, X_test, k=3):
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(X_train, y_train)
    return model.predict(X_test)

def predict_with_xgboost(X_train, y_train, X_test): #(클래스가 1부터 시작해서 수정)
    model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
    y_train_shifted = y_train - 1
    model.fit(X_train, y_train_shifted)
    return model.predict(X_test)

def predict_with_lstm(X_train, y_train, X_test):
    # LSTM 모델 정의
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(X_train.shape[1], 1)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # 데이터 차원 변경
    X_train_reshaped = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_test_reshaped = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

    model.fit(X_train_reshaped, y_train, epochs=50, batch_size=32, verbose=0)
    return (model.predict(X_test_reshaped) > 0.5).astype(int)

def predict_with_cnn(X_train, y_train, X_test):
    # CNN 모델 정의
    model = Sequential()
    model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # 데이터 차원 변경
    X_train_reshaped = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_test_reshaped = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

    model.fit(X_train_reshaped, y_train, epochs=50, batch_size=32, verbose=0)
    return (model.predict(X_test_reshaped) > 0.5).astype(int)

In [55]:
# Define predictions dictionary
predictions = {
    "Logistic Regression": predict_with_logistic_regression(X_synthetic, y_synthetic, X_test),
    "CART": predict_with_cart(X_synthetic, y_synthetic, X_test),
    "KNN": predict_with_knn(X_synthetic, y_synthetic, X_test),
    "XGBoost": predict_with_xgboost(X_synthetic, y_synthetic, X_test),
    "LSTM": predict_with_lstm(X_synthetic, y_synthetic, X_test),
    "CNN": predict_with_cnn(X_synthetic, y_synthetic, X_test)
}

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [05:52:34] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(*

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 167ms/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step


In [56]:
# 성능 지표 저장을 위한 리스트
results = []
for model_name, pred_y in predictions.items():
    accuracy = accuracy_score(y_test, pred_y)
    # average 매개변수를 'macro'로 변경(수정 이진분류를 기준으로 생성된 거라 당중분류를 위해서 옵션 변경)
    recall = recall_score(y_test, pred_y, average='macro')
    f1 = f1_score(y_test, pred_y, average='macro')
    specificity = specificity_score(y_test, pred_y, average='macro')
    conf_matrix = confusion_matrix(y_test, pred_y)
    results.append([accuracy, f1, recall, specificity, conf_matrix])

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [57]:
from collections import Counter

# 증강 전 클래스별 샘플 수
original_class_distribution = Counter(y)
print("증강 전 클래스별 샘플 수:")
for cls, count in original_class_distribution.items():
    print(f"클래스 {cls}: {count}개")

# 증강 후 클래스별 샘플 수
augmented_class_distribution = Counter(y_synthetic)
print("\n증강 후 클래스별 샘플 수:")
for cls, count in augmented_class_distribution.items():
    print(f"클래스 {cls}: {count}개")

증강 전 클래스별 샘플 수:
클래스 1: 76개
클래스 2: 31개
클래스 3: 25개
클래스 4: 32개
클래스 5: 17개

증강 후 클래스별 샘플 수:
클래스 1: 76개
클래스 2: 71개
클래스 3: 71개
클래스 4: 71개
클래스 5: 71개


In [58]:
# 성능 지표를 DataFrame으로 변환
results_df = pd.DataFrame(results, columns=["Accuracy", "F1", "Recall", "Specificity", "Confusion Matrix"], index=predictions.keys())
model_results = results_df.T
# 결과를 출력
print("\n모델 성능 비교 결과:")
print(model_results)

# 결과를 CSV 파일로 저장
model_results.to_csv("/content/drive/My Drive/PhalangesOutlinesCorrect/results/msmote(다중)_models_worms_result.csv")


모델 성능 비교 결과:
                                                Logistic Regression  \
Accuracy                                                   0.233766   
F1                                                         0.224318   
Recall                                                     0.244639   
Specificity                                                0.806818   
Confusion Matrix  [[8, 6, 4, 9, 6], [3, 2, 3, 3, 2], [1, 2, 5, 0...   

                                                               CART  \
Accuracy                                                   0.324675   
F1                                                         0.260286   
Recall                                                     0.268217   
Specificity                                                0.824946   
Confusion Matrix  [[15, 5, 4, 7, 2], [7, 1, 1, 4, 0], [1, 1, 3, ...   

                                                                KNN  \
Accuracy                                                   0.